In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
from skimage.color import gray2rgb
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

# Function to extract Mel spectrograms from audio files
def extract_mel_spectrogram(audio_file, n_mels=64, hop_length=512):
    y, sr = librosa.load(audio_file, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    return mel_spectrogram

# Path to your bird audio dataset
dataset_path = "birds_audio_dataset/"

# List of supported audio file extensions
supported_extensions = ['.wav', '.mp3', '.mp4']

# Load the trained MobileNet model
model = load_model("mobilenet_model.h5", compile=False)

# Compile the model with eager execution
model.compile(run_eagerly=True)

# Preprocessing function for test data
def preprocess_test_data(dataset_path, supported_extensions):
    test_data = []
    labels = []

    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):
            for audio_file in os.listdir(class_path):
                file_extension = os.path.splitext(audio_file)[1].lower()  # Get the file extension
                if file_extension in supported_extensions:
                    audio_path = os.path.join(class_path, audio_file)
                    try:
                        mel_spec = extract_mel_spectrogram(audio_path)
                       # Resize spectrogram to match MobileNetV2 input size
                        resized_spec = resize(mel_spec, (64, 64), anti_aliasing=True)
                        # Convert grayscale spectrogram to RGB
                        rgb_spec = gray2rgb(resized_spec)
                        test_data.append(rgb_spec)
                        labels.append(class_name)
                    except Exception as e:
                        print(f"Error processing {audio_path}: {e}")

    test_data = np.array(test_data)
    labels = np.array(labels)
    return test_data, labels

# Preprocess the test data
X_test, y_test = preprocess_test_data(dataset_path, supported_extensions)

# Evaluate the model
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Decode labels
label_encoder = LabelEncoder()
label_encoder.fit(y_test)
y_test_encoded = label_encoder.transform(y_test)

# Calculate accuracy
accuracy = np.mean(predicted_labels == y_test_encoded)
print("Accuracy:", accuracy)
